In [2]:
import os
import sys

import pandas as pd
import numpy as np
## from pyhive import hive
import threshold_build_service as tbs

# %% Import and Pre-process Data

# Read csv data
df = pd.read_csv('5g_home_web_session_duration_seconds_10_250.csv')

C:\Users\sindh\Desktop\web_latest\threshold_build_service.py:20: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\sindh\Desktop\web_latest\threshold_build_service.py:21: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\sindh\Desktop\web_latest\threshold_build_service.py:25: UserWarning:


The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`



In [5]:
df = pd.read_csv('5g_home_web_session_duration_seconds_10_250.csv')

# Filter for a single metric
df = df.loc[df.metricname == 'session_duration_seconds']

# Filter for the appropriate typology
df = df.loc[df.primary_high_level_reason == 'all']

# Rename the main column
df.rename(columns={'value':'session_duration_seconds'}, inplace=True)

# Remove nulls
print(len(df))
df = df.loc[df.metricname.notnull()]
print(len(df))

# %% Instantiate Metadata (class object for threshold build)
session_duration_seconds_object = tbs.Threshold_Build(
    metricname = 'session_duration_seconds', 
    channel = '5g_home', 
    input_df = df.copy(deep=True)
)

# %% Analyze exisiting relationship: Plot Bivariate
# bins = [i for i in range(-7200,-9,60)]

session_duration_seconds_object.plot_churn(
    bins = [i for i in range(11,250,1)]
)

240
240


In [6]:
# %% Explicitly record the relationship to churn
session_duration_seconds_object.relationship_to_churn = 'negative'

session_duration_seconds_object.input_df['session_duration_seconds'] = session_duration_seconds_object.input_df['session_duration_seconds'] * (1)

# %% Split the input dataframe
session_duration_seconds_object.split = (11, None)

# %% Create universe of potential thresholds

session_duration_seconds_object.create_3(
    list_of_thresholds = [i for i in range(11, 250, 1)]
)

# %% Establish cutoffs for each potential threshold
session_duration_seconds_object.create_1_5(
    ## think of argument name for how many percentiles to loop through
)

session_duration_seconds_object.create_2_4(
    ## create argument for how many potential 2s and 4s we cycle through
)


# %% Create bins
session_duration_seconds_object.create_bins(
        input_df = session_duration_seconds_object.cutoff_universe_df,
        bin_definitions = {
                'bin_1':['1','btwn_1_2'],
                'bin_2':['2','btwn_2_3'],
                'bin_3':['3','btwn_3_4'],
                'bin_4':['4','btwn_4_5','5']
        }
)

# %% Validity Checks
session_duration_seconds_object.cutoff_validity_checks(
    num_bins = 4,
    ordered_bin_suffixes = [
        'bin_1',
        'bin_2',
        'bin_3',
        'bin_4'], 
    check_monotonic = True, 
    check_pop_size = True,
    min_pop_size = (sum(session_duration_seconds_object.input_df.events) * 0.01)
)

# Looking at the number of valid curves
print('Total Completely Valid Cutoffs:',
    sum((session_duration_seconds_object.cutoff_universe_df['is_monotonic'] == 'Y') &
    (session_duration_seconds_object.cutoff_universe_df['is_valid_bin_size'] == 'Y')),
    '\nTotal Monotonic Cutoffs:',
    sum((session_duration_seconds_object.cutoff_universe_df['is_monotonic'] == 'Y')),
    '\nTotal Cutoffs with Valid Bin Sizes:',
    sum((session_duration_seconds_object.cutoff_universe_df['is_valid_bin_size'] == 'Y'))
)


# %% Define ranking columns for shortlisting the best curves
print('Ranking using good vs bad')
session_duration_seconds_object.define_ranking(
    ordered_bin_suffixes = [
        'bin_1',
        'bin_2',
        'bin_3',
        'bin_4'],
    ranking_formula = 'good_v_bad',
    ranking_column = 'good_v_bad_rank',
    ranking_method = 'dense',
    rounding_factor = 3
)

print('Ranking using dif between extremes')
session_duration_seconds_object.define_ranking(
    ordered_bin_suffixes = [
        'bin_1',
        'bin_2',
        'bin_3',
        'bin_4'],
    ranking_formula = 'dif_in_extremes',
    ranking_column = 'dif_in_extremes_rank',
    ranking_method = 'dense',
    rounding_factor = 4
)

print('Ranking using pairwise ratio')
session_duration_seconds_object.define_ranking(
    ordered_bin_suffixes = [
        'bin_1',
        'bin_2',
        'bin_3',
        'bin_4'],
    ranking_formula = 'pairwise_ratio',
    ranking_column = 'pairwise_ratio_rank',
    ranking_method = 'dense',
    rounding_factor = 1
)

print('Ranking using sum of square errors')
session_duration_seconds_object.define_ranking(
    ordered_bin_suffixes = [
        'bin_1',
        'bin_2',
        'bin_3',
        'bin_4'],
    ranking_formula = 'sum_square_errors',
    ranking_column = 'sum_square_errors_rank',
    ranking_method = 'dense',
    rounding_factor = None      # Does not round the squared errors
)

# %% Saving curve dataframe to csv for inspection and manual curve selection
session_duration_seconds_object.cutoff_universe_df.sort_values(
    by = [
        'is_monotonic',
        'is_valid_bin_size',
        'pairwise_ratio_rank',
        'dif_in_extremes_rank',
        'sum_square_errors_rank'
    ],
    ascending = [
        False,
        False,
        True,
        True,
        True
    ],
    inplace = True
)

# Save sorted dataframe as csv
session_duration_seconds_object.cutoff_universe_df.to_csv('session_duration_seconds_cutoffs.csv')

Resetting result_table
Cutoff percent: 0.01
Cutoff percent: 0.02
Cutoff percent: 0.03
Cutoff percent: 0.04
Cutoff percent: 0.05
Cutoff percent: 0.06
Cutoff percent: 0.07
Cutoff percent: 0.08
Cutoff percent: 0.09
Cutoff percent: 0.1
Threshold: 11
No possible cutoffs to choose from for cutoff 2
Size of filtered cartesian df: 0
No cutoffs found that were monotonic between 2 and 4 for threshold: 11
Monotonic cutoffs for 4 but not 2 were found for threshold: 11
Threshold: 12
No possible cutoffs to choose from for cutoff 2
Size of filtered cartesian df: 0
No cutoffs found that were monotonic between 2 and 4 for threshold: 12
Monotonic cutoffs for 4 but not 2 were found for threshold: 12
Threshold: 13
Size of filtered cartesian df: 0
No cutoffs found that were monotonic between 2 and 4 for threshold: 13
We have valid 2s and 4s, but no monotonicty between them for threshold: 13
Threshold: 14
Size of filtered cartesian df: 0
No cutoffs found that were monotonic between 2 and 4 for threshold: 14

Big table is not empty... Appending
New size of big table: 102300
This condition picked up completely valid cutoffs
Threshold: 59
Size of filtered cartesian df: 4402
Big table is not empty... Appending
New size of big table: 107000
This condition picked up completely valid cutoffs
Threshold: 60
Size of filtered cartesian df: 4501
Big table is not empty... Appending
New size of big table: 111800
This condition picked up completely valid cutoffs
Threshold: 61
Size of filtered cartesian df: 3422
Big table is not empty... Appending
New size of big table: 116700
This condition picked up completely valid cutoffs
Threshold: 62
Size of filtered cartesian df: 2366
Big table is not empty... Appending
New size of big table: 121700
This condition picked up completely valid cutoffs
Threshold: 63
Size of filtered cartesian df: 1743
Big table is not empty... Appending
New size of big table: 126800
This condition picked up completely valid cutoffs
Threshold: 64
Size of filtered cartesian df: 2239
Big 

Big table is not empty... Appending
New size of big table: 459800
This condition picked up completely valid cutoffs
Threshold: 109
Size of filtered cartesian df: 1828
Big table is not empty... Appending
New size of big table: 469500
This condition picked up completely valid cutoffs
Threshold: 110
Size of filtered cartesian df: 1290
Big table is not empty... Appending
New size of big table: 479300
This condition picked up completely valid cutoffs
Threshold: 111
Size of filtered cartesian df: 945
Big table is not empty... Appending
New size of big table: 489200
This condition picked up completely valid cutoffs
Threshold: 112
Size of filtered cartesian df: 1413
Big table is not empty... Appending
New size of big table: 499200
This condition picked up completely valid cutoffs
Threshold: 113
Size of filtered cartesian df: 3831
Big table is not empty... Appending
New size of big table: 509200
This condition picked up completely valid cutoffs
Threshold: 114
Size of filtered cartesian df: 6223

Big table is not empty... Appending
New size of big table: 874700
This condition picked up completely valid cutoffs
Threshold: 158
Size of filtered cartesian df: 3168
Big table is not empty... Appending
New size of big table: 883700
This condition picked up completely valid cutoffs
Threshold: 159
Size of filtered cartesian df: 5682
Big table is not empty... Appending
New size of big table: 892600
This condition picked up completely valid cutoffs
Threshold: 160
Size of filtered cartesian df: 5520
Big table is not empty... Appending
New size of big table: 901400
This condition picked up completely valid cutoffs
Threshold: 161
Size of filtered cartesian df: 4590
Big table is not empty... Appending
New size of big table: 910100
This condition picked up completely valid cutoffs
Threshold: 162
Size of filtered cartesian df: 3846
Big table is not empty... Appending
New size of big table: 918700
This condition picked up completely valid cutoffs
Threshold: 163
Size of filtered cartesian df: 204

Big table is not empty... Appending
New size of big table: 1172400
This condition picked up completely valid cutoffs
Threshold: 207
Size of filtered cartesian df: 488
Big table is not empty... Appending
New size of big table: 1176500
This condition picked up completely valid cutoffs
Threshold: 208
Size of filtered cartesian df: 360
Big table is not empty... Appending
New size of big table: 1180500
This condition picked up completely valid cutoffs
Threshold: 209
Size of filtered cartesian df: 227
No cutoffs found that were monotonic between 2 and 4 for threshold: 209
We have valid 2s and 4s, but no monotonicty between them for threshold: 209
Threshold: 210
Size of filtered cartesian df: 227
No cutoffs found that were monotonic between 2 and 4 for threshold: 210
We have valid 2s and 4s, but no monotonicty between them for threshold: 210
Threshold: 211
Size of filtered cartesian df: 227
Big table is not empty... Appending
New size of big table: 1184200
This condition picked up completely 

Ranking using dif between extremes
Ranking using pairwise ratio
Ranking using sum of square errors


In [7]:
# %% Establish best threshold (pick final candidate)

# Top row is the winner
best_session_duration_seconds_curve = session_duration_seconds_object.cutoff_universe_df.iloc[0,:]

#%%
# Assign cutoffs from the winner to object attributes
session_duration_seconds_object.best_threshold = best_session_duration_seconds_curve['threshold'] * (1)
print('threshold:', session_duration_seconds_object.best_threshold)

session_duration_seconds_object.best_cutoff_1 =  best_session_duration_seconds_curve['cutoff_1'] * (1)
print('cutoff 1:', session_duration_seconds_object.best_cutoff_1)

session_duration_seconds_object.best_cutoff_2 =  best_session_duration_seconds_curve['cutoff_2'] * (1)
print('cutoff 2:', session_duration_seconds_object.best_cutoff_2)

session_duration_seconds_object.best_cutoff_4 =  best_session_duration_seconds_curve['cutoff_4'] * (1)
print('cutoff 4:', session_duration_seconds_object.best_cutoff_4)

session_duration_seconds_object.best_cutoff_5 =  best_session_duration_seconds_curve['cutoff_5'] * (1)
print('cutoff 5:', session_duration_seconds_object.best_cutoff_5)

threshold: 29
cutoff 1: 11
cutoff 2: 12
cutoff 4: 234
cutoff 5: 249


In [8]:
print (best_session_duration_seconds_curve)

threshold                                      29
cutoff_1                                       11
cutoff_2                                       12
cutoff_4                                      234
cutoff_5                                      249
events_1                                     3714
events_btwn_1_2                                 0
events_2                                     3637
events_btwn_2_3                             43707
events_3                                     1895
events_btwn_3_4                            114554
events_4                                      195
events_btwn_4_5                              2795
events_5                                      417
churners_1                                    105
churners_btwn_1_2                               0
churners_2                                    100
churners_btwn_2_3                            1134
churners_3                                     43
churners_btwn_3_4                            2935
